## Add hexapod motions to the mount plots

Craig Lage  04-Jun-25

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import Figure
from matplotlib.dates import num2date
from matplotlib.ticker import FuncFormatter
import matplotlib.dates as mdates
import numpy as np
from scipy.optimize import minimize
from pandas import DataFrame
from dataclasses import dataclass
from zoneinfo import ZoneInfo
from lsst.daf.butler import Butler, DimensionRecord
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.efdUtils import makeEfdClient
from lsst.summit.utils.efdUtils import getEfdData
from lsst.summit.utils.utils import dayObsIntToString
from lsst_efd_client import EfdClient
from astropy.coordinates import EarthLocation
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
from astropy.time import Time, TimeDelta
import astropy.units as u

In [ ]:
@dataclass
class MountData:
    begin: Time
    end: Time
    azimuthData: DataFrame
    elevationData: DataFrame
    rotationData: DataFrame
    rotationTorques: DataFrame
    camhexData: DataFrame
    m2hexData: DataFrame
    includedPrePadding: float
    includedPostPadding: float
    expRecord: DimensionRecord | None


def getAzElRotHexDataForPeriod(
    client: EfdClient,
    begin: Time,
    end: Time,
    prePadding: float = 0,
    postPadding: float = 0,
    maxDeltaT: float = 1.0e-3,
) -> MountData:
    azimuthData = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        begin=begin,
        end=end,
        prePadding=prePadding,
        postPadding=postPadding,
    )
    elevationData = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        begin=begin,
        end=end,
        prePadding=prePadding,
        postPadding=postPadding,
    )
    rotationData = getEfdData(
        client,
        "lsst.sal.MTRotator.rotation",
        begin=begin,
        end=end,
        prePadding=prePadding,
        postPadding=postPadding,
    )
    rotationTorques = getEfdData(
        client,
        "lsst.sal.MTRotator.motors",
        begin=begin,
        end=end,
        prePadding=prePadding,
        postPadding=postPadding,
    )
    hexData = getEfdData(
        client,
        "lsst.sal.MTHexapod.application",
        begin=begin,
        end=end,
        prePadding=prePadding,
        postPadding=postPadding,
    )
    camhexData = hexData[hexData['salIndex'] == 1]
    m2hexData = hexData[hexData['salIndex'] == 2]

    def calcDeltaT(params, args):
        # This calculates the deltaT needed
        # to make the median(error) = 0
        [values, valTimes, demand, demTimes] = args
        [deltaT] = params
        demandInterp = np.interp(valTimes, demTimes + deltaT, demand)
        error = (values - demandInterp) * 3600
        value = abs(np.median(error))
        return value

    azValues = np.asarray(azimuthData["actualPosition"])
    azValTimes = np.asarray(azimuthData["actualPositionTimestamp"])
    azDemand = np.asarray(azimuthData["demandPosition"])
    azDemTimes = np.asarray(azimuthData["demandPositionTimestamp"])
    elValues = np.asarray(elevationData["actualPosition"])
    elValTimes = np.asarray(elevationData["actualPositionTimestamp"])
    elDemand = np.asarray(elevationData["demandPosition"])
    elDemTimes = np.asarray(elevationData["demandPositionTimestamp"])

    # Calculate the deltaT needed to drive the median(error) to zero
    args = [azValues, azValTimes, azDemand, azDemTimes]
    x0 = [0.0]
    result = minimize(calcDeltaT, x0, args=args, method="Powell", bounds=[(-maxDeltaT, maxDeltaT)])
    deltaTAz = result.x[0]

    args = [elValues, elValTimes, elDemand, elDemTimes]
    x0 = [0.0]
    result = minimize(calcDeltaT, x0, args=args, method="Powell", bounds=[(-maxDeltaT, maxDeltaT)])
    deltaTEl = result.x[0]

    azDemandInterp = np.interp(azValTimes, azDemTimes + deltaTAz, azDemand)
    elDemandInterp = np.interp(elValTimes, elDemTimes + deltaTEl, elDemand)

    azError = (azValues - azDemandInterp) * 3600
    elError = (elValues - elDemandInterp) * 3600

    rotValues = np.asarray(rotationData["actualPosition"])
    rotDemand = np.asarray(rotationData["demandPosition"])
    rotError = (rotValues - rotDemand) * 3600

    azimuthData["azError"] = azError
    elevationData["elError"] = elError
    rotationData["rotError"] = rotError

    mountData = MountData(
        begin, end, azimuthData, elevationData, rotationData, rotationTorques, camhexData,
        m2hexData, prePadding, postPadding, None
    )
    return mountData


In [ ]:
client = makeEfdClient()
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])

In [ ]:
start = Time("2025-08-04T23:06:13", scale='utc')
end = Time("2025-08-04T23:07:18", scale='utc')
mountData = getAzElRotHexDataForPeriod(client, start, end)

In [ ]:
fig, axs = plt.subplots(6, 3, figsize=(16,16))
plt.subplots_adjust(wspace=0.5, hspace=0.8)

axs[0][0].set_title("Rotator position")
mountData.rotationData['actualPosition'].plot(ax = axs[0][0])

for i in range(2):
    axs[i+1][0].set_title(f"Rotator torque{i}")
    mountData.rotationTorques[f'torque{i}'].plot(ax = axs[i+1][0])

axs[3][0].set_title("Azimuth position")
mountData.azimuthData['actualPosition'].plot(ax = axs[3][0])

axs[4][0].set_title("Elevation position")
mountData.elevationData['actualPosition'].plot(ax = axs[4][0])

axs[5][0].set_axis_off()

for i in range(6):
    axs[i][1].set_title(f"CamHex Position{i}")
    mountData.camhexData[f'position{i}'].plot(ax = axs[i][1])

for i in range(6):
    axs[i][2].set_title(f"M2Hex Position{i}")
    mountData.m2hexData[f'position{i}'].plot(ax = axs[i][2])

xticks = []
xticklabels = []
for n in range(8):
    tick = (start + TimeDelta(10.0 * n, format='sec')).isot
    xticks.append(tick)
    xticklabels.append(tick.split('T')[1].split('.')[0])

for i in range(3):
    for j in range(6):
        if i == 0 and j > 2:
            continue
        axs[j][i].set_xticks(xticks)
        axs[j][i].set_xticklabels(xticklabels)
"""
for i in range(3):
    for j in range(6):
        if i == 0 and j > 2:
            continue
        for n in range(6):
            axs[j][i].axvline((start +  TimeDelta(0.3, format='sec')\
                               + TimeDelta(0.6 * n, format='sec')).isot, ls='--', color='k')
"""
saveFilename = f"/home/c/cslage/u/MTMount/mount_plots/Shutter_Noise_4_04Aug25.png"
plt.savefig(saveFilename)

In [ ]:
start = Time("2025-08-04T23:06:33", scale='utc')
end = Time("2025-08-04T23:06:43", scale='utc')
mountData = getAzElRotHexDataForPeriod(client, start, end)

In [ ]:
fig, axs = plt.subplots(2,1, figsize=(8,8))
plt.subplots_adjust(wspace=0.5, hspace=0.8)

for m, i in enumerate([0, 4]):
    axs[m].set_title(f"M2Hex Position{i}")
    mountData.m2hexData[f'position{i}'].plot(ax = axs[m])

xticks = []
xticklabels = []
for n in range(11):
    tick = (start + TimeDelta(1.0 * n, format='sec')).isot
    xticks.append(tick)
    xticklabels.append(tick.split('T')[1].split('.')[0])

for j in range(2):
    axs[j].set_xticks(xticks)
    axs[j].set_xticklabels(xticklabels)

for j in range(2):
    for n in range(12):
        axs[j].axvline((start +  TimeDelta(0.50, format='sec')\
                           + TimeDelta(0.8 * n, format='sec')).isot, ls='--', color='k')

saveFilename = f"/home/c/cslage/u/MTMount/mount_plots/Shutter_Noise_3_04Aug25.png"
plt.savefig(saveFilename)

In [ ]:
start = Time("2025-08-05T21:21:00", scale='utc')
end = Time("2025-08-05T21:21:30", scale='utc')
mountData = getAzElRotHexDataForPeriod(client, start, end)

In [ ]:
fig, axs = plt.subplots(6, 3, figsize=(16,16))
plt.subplots_adjust(wspace=0.5, hspace=0.8)

axs[0][0].set_title("Rotator position")
mountData.rotationData['actualPosition'].plot(ax = axs[0][0])

for i in range(2):
    axs[i+1][0].set_title(f"Rotator torque{i}")
    mountData.rotationTorques[f'torque{i}'].plot(ax = axs[i+1][0])

axs[3][0].set_title("Azimuth position")
mountData.azimuthData['actualPosition'].plot(ax = axs[3][0])

axs[4][0].set_title("Elevation position")
mountData.elevationData['actualPosition'].plot(ax = axs[4][0])

axs[5][0].set_axis_off()

for i in range(6):
    axs[i][1].set_title(f"CamHex Position{i}")
    mountData.camhexData[f'position{i}'].plot(ax = axs[i][1])

for i in range(6):
    axs[i][2].set_title(f"M2Hex Position{i}")
    mountData.m2hexData[f'position{i}'].plot(ax = axs[i][2])

xticks = []
xticklabels = []
for n in range(4):
    tick = (start + TimeDelta(10.0 * n, format='sec')).isot
    xticks.append(tick)
    xticklabels.append(tick.split('T')[1].split('.')[0])

for i in range(3):
    for j in range(6):
        if i == 0 and j > 2:
            continue
        axs[j][i].set_xticks(xticks)
        axs[j][i].set_xticklabels(xticklabels)
"""
for i in range(3):
    for j in range(6):
        if i == 0 and j > 2:
            continue
        for n in range(6):
            axs[j][i].axvline((start +  TimeDelta(0.3, format='sec')\
                               + TimeDelta(0.6 * n, format='sec')).isot, ls='--', color='k')
"""
saveFilename = f"/home/c/cslage/u/MTMount/mount_plots/Shutter_Noise_7_04Aug25.png"
plt.savefig(saveFilename)

In [ ]:
start = Time("2025-08-05T21:21:10", scale='utc')
end = Time("2025-08-05T21:21:20", scale='utc')
mountData = getAzElRotHexDataForPeriod(client, start, end)

In [ ]:
fig, axs = plt.subplots(2,1, figsize=(8,8))
plt.subplots_adjust(wspace=0.5, hspace=0.8)

for m, i in enumerate([0, 4]):
    axs[m].set_title(f"M2Hex Position{i}")
    mountData.m2hexData[f'position{i}'].plot(ax = axs[m])

xticks = []
xticklabels = []
for n in range(10):
    tick = (start + TimeDelta(1.0 * n, format='sec')).isot
    xticks.append(tick)
    xticklabels.append(tick.split('T')[1].split('.')[0])

for j in range(2):
    axs[j].set_xticks(xticks)
    axs[j].set_xticklabels(xticklabels)
    
for j in range(2):
    for n in range(6):
        axs[j].axvline((start +  TimeDelta(0.50, format='sec')\
                           + TimeDelta(0.5 * n, format='sec')).isot, ls='--', color='k')

for j in range(2):
    for n in range(8):
        axs[j].axvline((start +  TimeDelta(5.30, format='sec')\
                           + TimeDelta(0.63 * n, format='sec')).isot, ls='--', color='k')

saveFilename = f"/home/c/cslage/u/MTMount/mount_plots/Shutter_Noise_8_04Aug25.png"
plt.savefig(saveFilename)